# Polarizability using automatic differentiation

Simple example for computing properties using (forward-mode)
automatic differentiation.
For a more classical approach and more details about computing polarizabilities,
see Polarizability by linear response.

In [1]:
using DFTK
using LinearAlgebra
using ForwardDiff

# Construct PlaneWaveBasis given a particular electric field strength
# Again we take the example of a Helium atom.
function make_basis(ε::T; a=10., Ecut=30) where {T}
    lattice=T(a) * I(3)  # lattice is a cube of $a$ Bohrs
    # Helium at the center of the box
    atoms     = [ElementPsp(:He, psp=load_psp("hgh/lda/He-q2"))]
    positions = [[1/2, 1/2, 1/2]]

    model = model_DFT(lattice, atoms, positions, [:lda_x, :lda_c_vwn];
                      extra_terms=[ExternalFromReal(r -> -ε * (r[1] - a/2))],
                      symmetries=false)
    PlaneWaveBasis(model; Ecut, kgrid=[1, 1, 1])  # No k-point sampling on isolated system
end

# dipole moment of a given density (assuming the current geometry)
function dipole(basis, ρ)
    @assert isdiag(basis.model.lattice)
    a  = basis.model.lattice[1, 1]
    rr = [a * (r[1] - 1/2) for r in r_vectors(basis)]
    sum(rr .* ρ) * basis.dvol
end

# Function to compute the dipole for a given field strength
function compute_dipole(ε; tol=1e-8, kwargs...)
    scfres = self_consistent_field(make_basis(ε; kwargs...); tol)
    dipole(scfres.basis, scfres.ρ)
end;

With this in place we can compute the polarizability from finite differences
(just like in the previous example):

In [2]:
polarizability_fd = let
    ε = 0.01
    (compute_dipole(ε) - compute_dipole(0.0)) / ε
end

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -2.770691488123                   -0.53    8.0         
  2   -2.772133317956       -2.84       -1.30    1.0    171ms
  3   -2.772169216987       -4.44       -2.65    1.0    215ms
  4   -2.772170719038       -5.82       -3.90    2.0    203ms
  5   -2.772170722797       -8.42       -4.42    2.0    239ms
  6   -2.772170723007       -9.68       -5.49    1.0    191ms
  7   -2.772170723015      -11.10       -5.96    2.0    214ms
  8   -2.772170723015      -12.83       -6.65    2.0    263ms
  9   -2.772170723015      -14.24       -7.51    1.0    218ms
 10   -2.772170723015   +    -Inf       -8.33    2.0    255ms
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -2.770779647124                   -0.52    9.0         
  2   -2.772060093959       -2.89       -1.32    1.0    217ms
  3   -2.7

1.7735580620102331

We do the same thing using automatic differentiation. Under the hood this uses
custom rules to implicitly differentiate through the self-consistent
field fixed-point problem.

In [3]:
polarizability = ForwardDiff.derivative(compute_dipole, 0.0)
println()
println("Polarizability via ForwardDiff:       $polarizability")
println("Polarizability via finite difference: $polarizability_fd")

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -2.770744112124                   -0.53    9.0         
  2   -2.772053395463       -2.88       -1.30    1.0    173ms
  3   -2.772082778211       -4.53       -2.68    1.0    228ms
  4   -2.772083415071       -6.20       -3.84    2.0    207ms
  5   -2.772083417584       -8.60       -4.35    2.0    200ms
  6   -2.772083417805       -9.65       -5.74    1.0    239ms
  7   -2.772083417811      -11.28       -6.03    2.0    207ms
  8   -2.772083417811      -13.08       -6.73    1.0    187ms
  9   -2.772083417811      -14.57       -7.43    1.0    240ms
 10   -2.772083417811   +  -14.45       -8.34    2.0    232ms

Polarizability via ForwardDiff:       1.772534983150897
Polarizability via finite difference: 1.7735580620102331
